<a href="https://colab.research.google.com/gist/opal16078/f2bdb3670793fe3d1a22b2998d13df07/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv('house_data_train.csv', index_col = 0)
df = df.dropna()
df1 = df.head(16211)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression , Lasso , Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


X = df1['sqft_living'].values
y = df1['price'].values

X_train , X_test , y_train , y_test = train_test_split(X.reshape(-1,1),y,test_size = 0.3, shuffle=False)

steps = [("imp_mean", SimpleImputer()) , 
         ("scaler", StandardScaler()) , 
         ("transform", PolynomialFeatures(degree = 7,include_bias=False)) , 
         ("LinearReg" , LinearRegression())
        #  ("LassoReg" , Lasso(alpha = 100 , max_iter = 20000 , normalize = True))
        #  ("RidgeReg" , Ridge(alpha = 1 , max_iter = 20000 , normalize = True))
         ]

pipeline = Pipeline(steps)
pipeline.fit(X_train,y_train)
rscore_test = pipeline.score(X_test,y_test)
rscore_train = pipeline.score(X_train,y_train)
y_pred = pipeline.predict(X_test)

print('r2-score-test:', rscore_test.round(2) , '\nr2-score-train:', rscore_train.round(2))


r2-score-test: 0.58 
r2-score-train: 0.55


In [ ]:
kfold = KFold(n_splits = 10 , shuffle=True , random_state= 10)
steps = [("imp_mean", SimpleImputer()) , 
         ("scaler", StandardScaler()) , 
         ("transform", PolynomialFeatures(degree = 5,include_bias=False)) , 
         ("LinearReg" , LinearRegression())
         ]
pipeline = Pipeline(steps)
pipeline.fit(X_train,y_train)
cv_score = cross_val_score( pipeline , X_train , y_train , cv=kfold )

print("cv_score",cv_score,"\ncv_score_mean",cv_score.mean())


cv_score [0.59325715 0.44910004 0.55396046 0.52830363 0.50981497 0.49758716
 0.58792438 0.51983219 0.60847486 0.52593038] 
cv_score_mean 0.5374185215072356


In [ ]:
import joblib

joblib.dump( pipeline ,'best_model.pkl')


['best_model.pkl']